In [14]:
from moviepy.editor import VideoFileClip, vfx, ImageSequenceClip
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import random

In [51]:
def convert_to_30fps(input_video_path, output_video_path):
    video = VideoFileClip(input_video_path)
    # new_video = video.set_fps(30)
    video.write_videofile(output_video_path, fps=30.0, codec='libx264')

def rotate_to_landscape(input_video_path, output_video_path, degrees=90):
    video = VideoFileClip(input_video_path)
    # Check if the video is in portrait mode (height > width)
    if video.size[1] > video.size[0]:
        # Rotate if in portrait
        rotated_video = video.rotate(degrees)
        rotated_video.write_videofile(output_video_path, codec='libx264')
    else:
        # No need to rotate
        print("Video is already in landscape mode")
        video.write_videofile(output_video_path, codec='libx264')

# Example usage:
convert_to_30fps('C:/Users/a/Videos/Music/20200826_193357.mp4', 'test_video2.mp4')
# rotate_to_landscape('test_video1.mp4', 'test_video1.mp4')

Moviepy - Building video test_video2.mp4.
MoviePy - Writing audio in test_video2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video test_video2.mp4



Moviepy - Done !
Moviepy - video ready test_video2.mp4


In [56]:
def crop_video(input_video_path, output_video_path, start_time, duration=5):
    video = VideoFileClip(input_video_path)
    # Calculate end time based on start time and duration
    end_time = start_time + duration
    # Create the subclip
    cropped_video = video.subclip(start_time, end_time)
    cropped_video.write_videofile(output_video_path, codec='libx264', fps=video.fps)

# Example usage:
crop_video('test_video2.mp4', 'cropped_test_video2.mp4', start_time=5)  # Start at 10 seconds

Moviepy - Building video cropped_test_video2.mp4.
MoviePy - Writing audio in cropped_test_video2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video cropped_test_video2.mp4



Moviepy - Done !
Moviepy - video ready cropped_test_video2.mp4


In [57]:
def convert_to_grayscale(input_video_path, output_video_path):
    video = VideoFileClip(input_video_path)
    # Apply the black and white effect
    grayscale_video = video.fx(vfx.blackwhite)
    # Write the resulting video to file
    grayscale_video.write_videofile(output_video_path, codec='libx264', fps=video.fps)

# Example usage:
convert_to_grayscale('cropped_test_video2.mp4', 'greyscale_cropped_test_video2.mp4')

Moviepy - Building video greyscale_cropped_test_video2.mp4.
MoviePy - Writing audio in greyscale_cropped_test_video2TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video greyscale_cropped_test_video2.mp4



Moviepy - Done !
Moviepy - video ready greyscale_cropped_test_video2.mp4


In [54]:
video = VideoFileClip("greyscale_cropped_test_video1.mp4")
video.size[0], video.size[1]

(2560, 1440)

In [61]:
def average_circle_pixels(input_video_path, center, radius, fps=30, progressbar=True):
    video = VideoFileClip(input_video_path)
    frames = list(video.iter_frames(fps=fps, dtype='uint8'))
    height, width = frames[0].shape[:2]

    # Check if the circle is within the video's boundaries
    x, y = center
    if not (radius <= x <= width - radius and radius <= y <= height - radius):
        raise ValueError("Circle is out of the video's boundaries.")

    # Prepare to extract pixels
    avg_values = []
    xx, yy = np.meshgrid(np.arange(width), np.arange(height))
    mask = (xx - x)**2 + (yy - y)**2 <= radius**2

    # Process each frame
    if progressbar:
        for frame in tqdm(frames, desc="Processing frames"):
            gray_frame = np.dot(frame[...,:3], [0.2989, 0.5870, 0.1140])  # Convert to grayscale
            circle_pixels = gray_frame[mask]
            avg_values.append(np.mean(circle_pixels))
    else:
        for frame in frames:
            gray_frame = np.dot(frame[...,:3], [0.2989, 0.5870, 0.1140])  # Convert to grayscale
            circle_pixels = gray_frame[mask]
            avg_values.append(np.mean(circle_pixels))

    return avg_values

In [ ]:
# Example usage:
center = (1000, 1000)  # x, y coordinates of the circle's center
radius = 100  # radius of the circle in pixels
average_values = average_circle_pixels('greyscale_cropped_test_video1.mp4', center, radius)

plt.figure()
plt.plot(average_values)
plt.show()

In [15]:
def reconstruct_video_with_circle(input_video_path, center, radius, grayscale_values, output_video_path, fps=30):
    video = VideoFileClip(input_video_path)
    frames = list(video.iter_frames(fps=fps, dtype='uint8'))
    height, width = frames[0].shape[:2]

    # Initialize an empty array for the new frames
    new_frames = []

    # Create the mask for the circle
    xx, yy = np.meshgrid(np.arange(width), np.arange(height))
    mask = (xx - center[0])**2 + (yy - center[1])**2 <= radius**2

    # Construct each frame
    for value in grayscale_values:
        frame = np.zeros((height, width), dtype=np.uint8)
        frame[mask] = value
        new_frames.append(np.stack([frame, frame, frame], axis=-1))  # Convert single channel to three-channel grayscale

    # Create a video clip from these frames
    clip = ImageSequenceClip(new_frames, fps=fps)
    clip.write_videofile(output_video_path, codec='libx264')

# Example usage:
center = (150, 150)  # x, y coordinates of the circle's center
radius = 50  # radius of the circle in pixels
# Assume grayscale_values is a list of average grayscale values for each frame
reconstruct_video_with_circle('greyscale_cropped_test_video1.mp4', center, radius, average_values, 'reconstructed_greyscale_cropped_test_video1.mp4')

t:  20%|█▉        | 198/1000 [21:01<00:48, 16.47it/s, now=None]

Moviepy - Building video circletest_greyscale_cropped_test_video1.mp4.
Moviepy - Writing video circletest_greyscale_cropped_test_video1.mp4



t:  20%|█▉        | 198/1000 [21:02<00:48, 16.47it/s, now=None]

Moviepy - Done !
Moviepy - video ready circletest_greyscale_cropped_test_video1.mp4


In [16]:
# circle: dict with fields center, radius, grayscale_values
# circles: list of multiple circle

In [49]:
def reconstruct_video_with_circles(input_video_path, circles, output_video_path, fps=30):
    video = VideoFileClip(input_video_path)
    height, width = video.size
    num_frames = int(video.duration * fps)

    # Initialize an empty array for the new frames
    new_frames = [np.zeros((height, width, 3), dtype=np.uint8) for _ in range(num_frames)]

    # Process each circle
    for circle in tqdm(circles):
        center, radius, grayscale_values = circle["center"], circle["radius"], circle["grayscale_values"]
        # Create the mask for the circle
        xx, yy = np.meshgrid(np.arange(width), np.arange(height))
        mask = (xx - center[0])**2 + (yy - center[1])**2 <= radius**2

        # Apply the grayscale values to the specified region in each frame
        for i in range(num_frames):
            value = grayscale_values[i]
            new_frames[i][mask] = np.array([value, value, value], dtype=np.uint8)

    # Create a video clip from these frames
    clip = ImageSequenceClip(new_frames, fps=fps)
    clip.write_videofile(output_video_path, codec='libx264')

In [63]:
circles = []

input_vid_path = 'greyscale_cropped_test_video2.mp4'
radius = 100
output_vid_path = 'reconstructed_greyscale_cropped_test_video2.mp4'
n_circles = 30

pbar = tqdm(total=n_circles, desc="Processing")

# get some circles
while len(circles) < 30:
    x = random.random()*video.size[0]
    y = random.random()*video.size[1]
    try:
        grayscale_values = average_circle_pixels(input_vid_path, [x, y], radius, progressbar=False)
        circle = {
            "center": [x, y], 
            "radius": radius, 
            "grayscale_values": grayscale_values
            }
        circles.append(circle)
        pbar.update(1)
    except ValueError:
        continue

Processing:  13%|█▎        | 4/30 [01:03<06:53, 15.90s/it]

here


Processing:  17%|█▋        | 5/30 [01:22<07:09, 17.18s/it]

here


Processing:  40%|████      | 12/30 [03:16<04:46, 15.92s/it]

...

Processing:  63%|██████▎   | 19/30 [05:17<03:10, 17.34s/it]

here
here


Processing:  70%|███████   | 21/30 [05:55<02:41, 17.99s/it]

here


Processing: 100%|██████████| 30/30 [08:24<00:00, 16.28s/it]

In [64]:
reconstruct_video_with_circles(input_vid_path, circles, output_vid_path)

Processing: 100%|██████████| 30/30 [20:25<00:00, 16.28s/it]

Moviepy - Building video reconstructed_greyscale_cropped_test_video2.mp4.
Moviepy - Writing video reconstructed_greyscale_cropped_test_video2.mp4



Processing: 100%|██████████| 30/30 [20:28<00:00, 16.28s/it]

Moviepy - Done !
Moviepy - video ready reconstructed_greyscale_cropped_test_video2.mp4


In [35]:
circles

[{'center': [1842.7112154724846, 703.7844018159929],
  'radius': 100,
  'grayscale_values': [38.807208428557786,
   39.57690577712704,
   39.9865792914664,
   41.735202832861184,
   44.85180236814463,
   45.381940010185566,
   46.618789925836325,
   48.50127664321864,
   50.512056860935154,
   50.81612932488779,
   51.55132651112455,
   52.16739629181654,
   54.719358022726546,
   53.77569583346595,
   55.99411355953782,
   56.97768645319413,
   58.374497453607916,
   61.73966365661902,
   61.03737529681383,
   61.62594679313747,
   63.5695726358341,
   68.48508192698222,
   69.61467580290926,
   69.63001628099438,
   67.1668510933571,
   66.58566339561385,
   69.27260223764205,
   67.14508161823215,
   62.25242391062164,
   57.95088383677626,
   54.80570390871184,
   52.93944881751917,
   50.01591015373842,
   50.92768197154406,
   51.64683849189928,
   53.023980580577394,
   54.22839907056689,
   53.06350936435687,
   51.01294574911672,
   49.711487595887576,
   48.43501341630327,
  